<a href="https://colab.research.google.com/github/1830668NabilahOshin/Numerical-methods-and-Neural-Network-Labwork/blob/main/seq2seq_model_LSTM_%26_GRU_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [ ]:
import numpy as np
import math
import pandas as pd
import torch
import torch.nn as nn
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#loading pretrained word2vec model

In [ ]:
import gensim.downloader as api
w2v_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


#Text Preprocessor

In [ ]:
class TextPreprocessor:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.punctuations = set(string.punctuation)

    def process_text(self, text):
        text = text.lower()
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'<.*?>', '', text)
        text = ''.join(char for char in text if char not in self.punctuations)
        tokens = word_tokenize(text)
        tokens = [token for token in tokens if token not in self.stop_words]
        text = ' '.join(tokens)

        return text

    def process_column(self, df, column_name):
        df[column_name] = df[column_name].apply(self.process_text)

        return df

#Loading dataset

In [ ]:
import csv

data = pd.read_csv('Clothes-Review.csv' , encoding='ISO-8859-1', header=0)


print(data.head())

   Unnamed: 0  Clothing ID  Age                    Title  \
0           0          767   33                      NaN   
1           1         1080   34                      NaN   
2           2         1077   60  Some major design flaws   
3           3         1049   50         My favorite buy!   
4           4          847   47         Flattering shirt   

                                         Review Text  Rating  Recommended IND  \
0  Absolutely wonderful - silky and sexy and comf...       4                1   
1  Love this dress!  it's sooo pretty.  i happene...       5                1   
2  I had such high hopes for this dress and reall...       3                0   
3  I love, love, love this jumpsuit. it's fun, fl...       5                1   
4  This shirt is very flattering to all due to th...       5                1   

   Positive Feedback Count   Division Name Department Name Class Name  
0                        0       Initmates        Intimate  Intimates  
1       

In [ ]:
print(data.columns)
data.columns = data.columns.str.strip()
dataf = data[['Review Text', 'Recommended IND']]
print(dataf.head())

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')
                                         Review Text  Recommended IND
0  Absolutely wonderful - silky and sexy and comf...                1
1  Love this dress!  it's sooo pretty.  i happene...                1
2  I had such high hopes for this dress and reall...                0
3  I love, love, love this jumpsuit. it's fun, fl...                1
4  This shirt is very flattering to all due to th...                1


In [ ]:
dataf = data[['Review Text', 'Recommended IND']]
print(dataf.head())



                                         Review Text  Recommended IND
0  Absolutely wonderful - silky and sexy and comf...                1
1  Love this dress!  it's sooo pretty.  i happene...                1
2  I had such high hopes for this dress and reall...                0
3  I love, love, love this jumpsuit. it's fun, fl...                1
4  This shirt is very flattering to all due to th...                1


#Applying preprocessor on Dataset

In [ ]:
preprocessor = TextPreprocessor()
data['Review Text'] = data['Review Text'].fillna('')
df = preprocessor.process_column(data, 'Review Text')

print(df.head())

   Unnamed: 0  Clothing ID  Age                    Title  \
0           0          767   33                      NaN   
1           1         1080   34                      NaN   
2           2         1077   60  Some major design flaws   
3           3         1049   50         My favorite buy!   
4           4          847   47         Flattering shirt   

                                         Review Text  Rating  Recommended IND  \
0        absolutely wonderful silky sexy comfortable       4                1   
1  love dress sooo pretty happened find store im ...       5                1   
2  high hopes dress really wanted work initially ...       3                0   
3  love love love jumpsuit fun flirty fabulous ev...       5                1   
4  shirt flattering due adjustable front tie perf...       5                1   

   Positive Feedback Count   Division Name Department Name Class Name  
0                        0       Initmates        Intimate  Intimates  
1       

#Function to get w2v vectors

In [ ]:
def w2v(sentence):
    tokenized_data = sentence.split()
    n_tokens = len(tokenized_data)
    if n_tokens >= 10:
        tokenized_data = tokenized_data[:10]
    else:
        pad_length = 10 - n_tokens
        tokenized_data += ["<EOS>"] * pad_length

    vectors = []
    for token in tokenized_data:
        if token in w2v_model:
            vec = w2v_model[token]
        else:
            vec = np.zeros(w2v_model.vector_size)
        vectors.append(vec)

    tensor = torch.stack([torch.tensor(vec, dtype=torch.float32) for vec in vectors])
    return tensor

#Defining Datasethelper child class

In [ ]:
class Datasethelper(Dataset):
  def __init__(self, df):
    super().__init__()
    self.data = df['Review Text'].values
    self.labels = df['Recommended IND'].values

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    text = self.data[index]
    label = self.labels[index]
    w2v_data = w2v(text)
    label = torch.tensor( label , dtype=torch.float32)
    #print(w2v_data.shape)
    #print(w2v_data.dtype)
    return w2v_data, label

#Splitting dataset + Preparing dataset by Dataloader

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_helper = Datasethelper(train_df)
test_helper = Datasethelper(test_df)


train_dloader = DataLoader(train_helper, batch_size = 12 , shuffle = True)
test_dloader = DataLoader(test_helper, batch_size = 12 , shuffle = False)

#Checking shapes

In [ ]:
for x,y in train_dloader:
  break
print(x.shape)
print(y.shape)
print(x.dtype)

torch.Size([12, 10, 300])
torch.Size([12])
torch.float32


========================================================  shuru

#LSTM based seq2seq Model

In [ ]:
class Seq2SeqLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2SeqLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.encoder = nn.LSTM(input_size, hidden_size)
        self.decoder = nn.LSTM(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        encoder_output, (encoder_hidden, encoder_cell) = self.encoder(x)
        decoder_output, _ = self.decoder(encoder_output, (encoder_hidden, encoder_cell))
        output = self.fc(decoder_output)
        output = output[:, 0, :]
        return output




#Initating before training

In [ ]:

input_size = 300
hidden_size = 128
output_size = 1
num_epochs = 10
learning_rate = 0.001

model = Seq2SeqLSTM(input_size, hidden_size, output_size)


criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



#Training

In [ ]:
total_step = len(train_dloader)
for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_correct = 0
    epoch_total = 0
    for i, (x, y) in enumerate(train_dloader):
        model.train()


        outputs = model(x)
        #print(outputs.shape)
        outputs = outputs.squeeze()
        loss = criterion(outputs, y)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        predicted_labels = torch.round(torch.sigmoid(outputs))
        correct = (predicted_labels == y).sum().item()
        total = y.size(0)
        accuracy = correct / total

        epoch_loss += loss.item()
        epoch_correct += correct
        epoch_total += total

    epoch_loss /= total_step
    epoch_accuracy = epoch_correct / epoch_total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

Epoch [1/10], Loss: 0.4062, Accuracy: 0.8384
Epoch [2/10], Loss: 0.4041, Accuracy: 0.8390
Epoch [3/10], Loss: 0.4029, Accuracy: 0.8397
Epoch [4/10], Loss: 0.4006, Accuracy: 0.8396
Epoch [5/10], Loss: 0.3990, Accuracy: 0.8415
Epoch [6/10], Loss: 0.3978, Accuracy: 0.8415
Epoch [7/10], Loss: 0.3963, Accuracy: 0.8429
Epoch [8/10], Loss: 0.3954, Accuracy: 0.8415
Epoch [9/10], Loss: 0.3942, Accuracy: 0.8423
Epoch [10/10], Loss: 0.3937, Accuracy: 0.8426


#Testing

In [ ]:

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_dloader:
        outputs = model(x)
        outputs = outputs.squeeze()
        predicted = torch.round(torch.sigmoid(outputs))
        total += y.size(0)
        correct += (predicted == y).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')


Test Accuracy: 82.18%


#GRU based seq2seq Model

In [ ]:
class Seq2SeqGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2SeqGRU, self).__init__()
        self.hidden_size = hidden_size

        self.encoder = nn.GRU(input_size, hidden_size)
        self.decoder = nn.GRU(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        encoder_output, encoder_hidden = self.encoder(x)
        decoder_output, _ = self.decoder(encoder_output, encoder_hidden)
        output = self.fc(decoder_output)
        output = output[:, 0, :]  # Select the first element along the second dimension
        return output


In [ ]:
# Define the hyperparameters
input_size1 = 300
hidden_size1 = 128
output_size1 = 1
num_epochs1 = 10
learning_rate1 = 0.001

# Initialize the model
model1 = Seq2SeqGRU(input_size1, hidden_size1, output_size1)

# Define the loss function and optimizer
criterion1 = nn.BCEWithLogitsLoss()
optimizer1 = torch.optim.Adam(model1.parameters(), lr=learning_rate)

In [ ]:
total_step = len(train_dloader)
for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_correct = 0
    epoch_total = 0
    for i, (x, y) in enumerate(train_dloader):
        model1.train()

        # Forward pass
        outputs = model1(x)
        outputs = outputs.squeeze()
        loss = criterion1(outputs, y)

        # Backward and optimize
        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()

        # Calculate accuracy
        predicted_labels = torch.round(torch.sigmoid(outputs))
        correct = (predicted_labels == y).sum().item()
        total = y.size(0)
        accuracy = correct / total

        epoch_loss += loss.item()
        epoch_correct += correct
        epoch_total += total

    epoch_loss /= total_step
    epoch_accuracy = epoch_correct / epoch_total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

Epoch [1/10], Loss: 0.4506, Accuracy: 0.8255
Epoch [2/10], Loss: 0.4349, Accuracy: 0.8287
Epoch [3/10], Loss: 0.4309, Accuracy: 0.8316
Epoch [4/10], Loss: 0.4280, Accuracy: 0.8312
Epoch [5/10], Loss: 0.4253, Accuracy: 0.8322
Epoch [6/10], Loss: 0.4233, Accuracy: 0.8338
Epoch [7/10], Loss: 0.4222, Accuracy: 0.8340
Epoch [8/10], Loss: 0.4200, Accuracy: 0.8348
Epoch [9/10], Loss: 0.4190, Accuracy: 0.8351
Epoch [10/10], Loss: 0.4175, Accuracy: 0.8355


In [ ]:
# Testing loop
model1.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_dloader:
        outputs = model1(x)
        outputs = outputs.squeeze()
        predicted = torch.round(torch.sigmoid(outputs))
        total += y.size(0)
        correct += (predicted == y).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 82.16%


#seq2seq with attention

In [ ]:
import torch.nn.functional as F

class Seq2SeqAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2SeqAttention, self).__init__()
        self.hidden_size = hidden_size

        self.encoder = nn.GRU(input_size, hidden_size)
        self.decoder = nn.GRU(hidden_size + input_size, hidden_size)  # Concatenate input with attention context
        self.attention = nn.Linear(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        encoder_output, encoder_hidden = self.encoder(x)

        decoder_hidden = encoder_hidden  # Initialize decoder hidden state with encoder final hidden state

        outputs = []
        for i in range(x.size(0)):
            context = self.attention(decoder_hidden[-1])  # Calculate attention context
            context = context.unsqueeze(0).expand_as(encoder_output)  # Expand context to match encoder output size

            attended_encoder_output = encoder_output * F.softmax(context, dim=2)  # Apply attention weights
            attended_encoder_output = torch.sum(attended_encoder_output, dim=0)  # Sum the attended encoder output

            decoder_input = torch.cat((x[i].unsqueeze(0), attended_encoder_output.unsqueeze(0)), dim=2)  # Concatenate input with attention context
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)

            output = self.fc(decoder_output)
            outputs.append(output)

        outputs = torch.cat(outputs, dim=0)
        return outputs
